In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

# Entrenamiento de modelos de prueba
from sklearn.linear_model import LinearRegression

# Evaluación de modelos de prueba
from sklearn.metrics import roc_auc_score
from sklearn.metrics import mean_squared_error

# Partición de train-test set
from sklearn.model_selection import train_test_split

# Crear datasets
from sklearn.datasets import make_regression

Vamos a crear un dataset sintetico utilizando las librerias de [Sklearn Datasets](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.make_regression.html), en particular nos interesa crear un problema de regresion
lineal al que podemos variarle sus parametros

In [ ]:
# Creamos un dataset de prueba
X, y = make_regression(n_samples = 1000,
                       n_features = 1,
                       noise = 2,
                       n_informative = 1,
                       random_state = 42)

new_data = np.append(X,y.reshape(-1,1),axis=1)
new_data.shape

In [ ]:
df = pd.DataFrame(new_data)

new_col = []
i = 0
for col in df.columns:
    if i ==  len(df.columns) - 1:
        new_col.append("target")
    else :
        new_col.append("feature_" + str(i+1))
        
    i += 1
    
df.columns = new_col

df.head()

In [ ]:
fig,axes = plt.subplots(2,figsize=(22,6))
axes[0].scatter(X,y)
sns.histplot(X, ax=axes[1]);

## Funciones auxiliares para generar datos anómalos

In [ ]:
def generate_nulls(df: pd.DataFrame,
                   cols: list = None,
                   percentage: float = 0.05) -> pd.DataFrame:
  """Con esta función vamos a poder generar nulos en ciertas columnas de nuestro
  dataset.
  """  
  cols = df.columns.tolist() if cols is None else cols
  seeds = np.random.randint(100, size = len(cols))
  
  nsamples = np.math.floor(len(df) * percentage)

  result = df.copy(deep = True)
  for i,col_name in enumerate(cols):
      np.random.seed(seeds[i])
      
      idx_to_remove = result[col_name].sample(n = nsamples).index
      result[col_name].iloc[idx_to_remove] = np.nan if result[col_name].dtype in [int,float] else None
  
  return result

def generate_outliers(df: pd.DataFrame,
                     cols: list = None,
                     extreme_outlier: bool = False,
                     percentage: float = 0.02) -> pd.DataFrame:
  """Con esta función vamos a poder generar outliers en ciertas columnas de nuestro
  dataset. Si le damos True a _extreme_outlier_ va a generar outliers con mucho
  peso en la regresión
  """  
  seeds = np.random.randint(100, size = len(df))
  
  nsamples = np.math.floor(len(df) * percentage)
  idx_to_change = df.sample(n = nsamples).index

  cols = df.columns.tolist() if cols is None else cols

  result = df.copy(deep = True)
  
  for i,col_name in enumerate(cols):
    np.random.seed(seeds[i])

    if extreme_outlier:
      iqr = result[col_name].quantile(0.75) - result[col_name].quantile(0.25)
      max_val = 3*iqr
      outlier_sign = [1 if np.random.random() < 0.5 else -1 for _ in range(nsamples)]
      outliers = np.multiply(outlier_sign, np.random.uniform(low = iqr,
                                                          high = max_val,
                                                          size = nsamples))
      
      result[col_name].loc[idx_to_change] = outliers
    else:
      samples = result[col_name].loc[idx_to_change].values
      np.random.shuffle(samples)
      result[col_name].iloc[idx_to_change] = samples
  return result

In [ ]:
df_outlier = generate_outliers(df,['target'], percentage = 0.01, extreme_outlier = True)

In [ ]:
sns.scatterplot(data = pd.concat([df.assign(dataset = 'set1'),
                                 df_outlier.assign(dataset = 'set2')]),
                x = 'feature_1',
                y = 'target',
                hue = 'dataset')

In [ ]:
columnas_a_modificar = ['feature_1'] 
df_missing = generate_nulls(df,columnas_a_modificar, percentage = 0.01)

In [ ]:
df_missing.isna().sum()

# Primera Parte

## Ejercicio 1

Crear una función que separe los datos en train-validation-test 70-20-10


Hints: 

* Usar Indexing y slicing
* Usar np.random.[...]

In [ ]:
def split(df: pd.DataFrame, train_size: float):

    return X_train, X_val, X_test, Y_train, Y_val, Y_test

In [ ]:
from sklearn.model_selection import train_test_split

X_train,X_test, Y_train, Y_test = train_test_split(df.feature_1,
                                                   df.target,
                                                   test_size = 0.3)

X_val, X_test, Y_val, Y_test = train_test_split(X_test,
                                               Y_test,
                                               train_size = xxx)

## Ejercicio 2

Utilizando la funcion `generate_nulls`. Insertar en la columna de features un 5% de valores nulos. Con esto, generar una función que les permita rellenar estos valores con la mediana, y otro con la media.

Compare los __scores__ obtenidos en las regresiones cuando tiene el dataset completo, y con los dos métodos de imputación. ¿Cambian mucho?

In [ ]:
df.feature_1.median()

In [ ]:
def mean_imputer(df: pd.DataFrame, columns: str) -> pd.DataFrame:
    pass
    

In [ ]:
def median_imputer(df: pd.DataFrame, columns: str) -> pd.DataFrame:
    pass
    

## Ejercicio 3

Los resultados del ejercicio 2 ¿Cómo se ven afectados cuando empezamos a a tener muchos más valores imputados? Pruebe con distintos porcentajes.

## Ejercicio 4

Utilizando la funcion `generate_outliers` generar puntos extremos dentro de los datos que generamos anteriormente. En este ejercicio dejar setteado `extreme_outliers` como `False` y observe como variando el porcentaje de los mismos la regresión comienza a afectarse.

## Ejercicio 5

Generar ahora valores extremos utilizando `extreme_outliers`. ¿Cómo afecta esto a la regresión? ¿Se observa algo interesante?


## Ejercicio 6

utilizando el archivo CSV `clase3v2.csv`, vamos a cargarlo como dataframe y deberemos:
 - Generar splits de training/test
 - Limpiar/Imputar datos Nulos
 - Entrenar un modelo de regresión multivariada
 - Reportar $R^2$ en test.

In [ ]:
data_happiness = pd.read_csv('clase3v2.csv', delimiter = ';')

## Ejercicio 7

Por último, manteniendo un dataset limpio como el que generamos previo al ejercio 1, utilizando `make_regression` de `scikit-learn`. Analizar que pasa si comenzamos a modificar el parametro `noise`

# Segunda parte


## Ejercicio 8

Vamos a tomar dos dataframes creados uno en el ejercicio 4 y otro del 5, utilizando el modelo `HuberRegressor` en `linear_model` de `scikit-learn`. Vamos a entrenar dos modelos robustos, el modelo mejora?

***Nota:*** Utilizar los valores por defecto del modelo

## Ejercicio 9 (Opcional)

Utilizando este esta funcion de Gradiente descendente de como idea, codear su implementación de Sthocastic Gradient Descent (SGD)

```python

def ssr_gradient(x, y, b):
    res = b[0] + b[1] * x - y
    return res.mean(), (res * x).mean()

def gradient_descent(
    x: list,
    y: list,
    start: list,
    learn_rate: float = 0.1,
    n_iter: int = 50,
    tolerance: float =1e-06
):
    vector = start
    for _ in range(n_iter):
        diff = -learn_rate * np.array(ssr_gradient(x, y, vector))
        if np.all(np.abs(diff) <= tolerance):
            break
        vector += diff
    return vector
```

# Tercera Parte

Vamos a crear un nuevo dataset, para ello vamos a usar el metodo `make_classification` de `sklearn`

In [ ]:
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression

X, y = make_classification(
    n_samples = 1000,
    n_features=2,
    n_redundant=0,
    n_informative=2,
    random_state=1,
    n_classes=2,
    n_clusters_per_class=1
)

## Ejercicio 10

Con el dataset creado para clasficación vamos a utilizar el modelo `LogisticRegression`. Vamos a entrenar modelos variando los parámetros en particular `tol` y `max_iter`.
¿Cómo cambian estas regresiones?


## Ejercicio 11

Vamos a utilizar ahora el dataset de [iris](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_iris.html). Utilizando `LogisticRegression` vamos a realizar un clasificador multiclases.
¿Cómo se compara con el modelo de `Perceptron`?

In [ ]:
from sklearn.datasets import load_iris
X,y = load_iris(return_X_y=True, as_frame=True)

HINT: La clase perceptron la pueden utilizar así

```python
from sklearn.linear_model import Perceptron
X, y = load_iris(return_X_y=True, as_frame=True)
clf = Perceptron(tol=1e-3, random_state=0)
clf.fit(X, y)

clf.score(X, y)
```